# 모듈, 라이브러리 준비

In [1]:
  
# SKT Brain의 KoBERT, 학습 및 테스트 데이터셋만 따로 준비 = 카카오 브런치[text, label]
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=f0b7eac751245d42084c288c9e18e6267ce35fe19dcbeccd0a128833d592d3c2
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.1 MB/s 
     |████████████████████████████████| 769 kB 7.4 MB/s 
     |████████████████████████████████| 3.0 MB 67.7 MB/s 
     |████████████████████████████████| 895 kB 85.7 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-y7hdlcxl
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

# 환경 세팅

In [2]:
# GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [3]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 준비

In [4]:
# 데이터셋 불러오기
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/데캡디/감성대화/Training/train.xlsx')

# 해당 column만 추출
df = data[ ['감정_대분류', '사람문장1'] ]

df.head()

,감정_대분류,사람문장1
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.


In [5]:
# 데이터 합치기
new_data = df[['사람문장1', '감정_대분류']]
new_data = new_data.rename(columns={'사람문장1':'content', '감정_대분류':'label'})
new_data.head()

,content,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,불안
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,당황
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기쁨


In [6]:
# label encoding
new_data.loc[(new_data['label'] == "기쁨"), 'label'] = 0  #발라드 => 0
new_data.loc[(new_data['label'] == "불안"), 'label'] = 1  #밤 => 0
new_data.loc[(new_data['label'] == "당황"), 'label'] = 2  #슬픈 => 0
new_data.loc[(new_data['label'] == "슬픔"), 'label'] = 3  #발라드 => 0
new_data.loc[(new_data['label'] == "분노"), 'label'] = 4  #밤 => 0
new_data.loc[(new_data['label'] == "상처"), 'label'] = 5  #슬픈 => 0
new_data.loc[(new_data['label'] == "불안 "), 'label'] = 1  #슬픈 => 0
new_data.loc[(new_data['label'] == "기쁨 "), 'label'] = 0  #슬픈 => 0

# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(new_data['content'], new_data['label'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [7]:
# 데이터 확인
print(data_list[0])
print(data_list[-1])

['아내가 드디어 출산하게 되어서 정말 신이 나.', '0']
['친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.', '1']


In [8]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [9]:
# 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

30659
10220


# KoBERT 모델 준비

In [10]:
# KoBERT모델의 입력으로 들어갈 수 있는 형태가 되도록 변환해주는 class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        # BERTSentenceTransform이라는 모듈 사용
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # 타입 변환
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # item 가져오기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 길이 출력
    def __len__(self):
        return (len(self.labels))

In [11]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 패딩
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# 데이터 확인 -> array1은 시퀀스, array2는 길이와 타입, array3은 attention mask 시퀀스(1로 패딩된 값들은 연산할 필요가 없기 때문에, 연산을 하지 않아도 됨 -> 이걸 알려주는 데이터가 바로 얘)
data_train[0]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


(array([   2, 4688, 6797, 5940,  950, 7795, 4384, 7088, 2573, 5794, 5439,
        5007, 5112, 5330, 1370,  517,   54,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 4)

In [15]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio) # 초기에 성능을 끌어올리는 것 = 운동할 때 워밍업 느낌

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] # 맞은 비율
    return train_acc
    
train_dataloader

In [18]:
# 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/480 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.9016027450561523 train acc 0.15625
epoch 1 batch id 201 loss 1.2466961145401 train acc 0.34530472636815923
epoch 1 batch id 401 loss 1.0841554403305054 train acc 0.4681265586034913
epoch 1 train acc 0.48976779513888885


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6546963778409091


  0%|          | 0/480 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.7200099229812622 train acc 0.796875
epoch 5 batch id 201 loss 0.8555232882499695 train acc 0.7828047263681592
epoch 5 batch id 401 loss 0.5752197504043579 train acc 0.79991427680798
epoch 5 train acc 0.8020833333333334


  0%|          | 0/160 [00:00<?, ?it/s]

epoch 5 test acc 0.6545010653409091


In [ ]:
'''
import matplotlib.pyplot as plt
plt.plot(np.arange(0.0, 7.0, 1.0), train_acc_list, np.arange(0.0, 7.0, 1.0), test_acc_list, 'r-')
'''

# 모델 평가

In [19]:
import numpy as np
import matplotlib.pyplot as plt
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [27]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

label_softmax = []
# 새로운 문장 테스트
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      #print("token_ids : ", token_ids)
      segment_ids = segment_ids.long().to(device)
      #print("segment_ids : ", segment_ids)

      valid_length= valid_length
      #print("valid_length : ", valid_length)
      label = label.long().to(device)
      #print("label : ", label)
      out = model(token_ids, valid_length, segment_ids)

      #print("out : ", out)

      test_eval=[]
      for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()
          logits = softmax(logits)
          print("softmax : ", logits)
          label_softmax = logits

          if np.argmax(logits) == 0:
                test_eval.append("기쁨")
          elif np.argmax(logits) == 1:
                test_eval.append("불안")
          elif np.argmax(logits) == 2:
                test_eval.append("당황")
          elif np.argmax(logits) == 3:
                test_eval.append("슬픔")
          elif np.argmax(logits) == 4:
                test_eval.append("분노")
          elif np.argmax(logits) == 5:
                test_eval.append("상처")

      print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
      print(str(label_softmax[0]) + "기쁨 " + str(label_softmax[1]) + "불안 " + str(label_softmax[2]) + "당황 " + str(label_softmax[3]) + "슬픔 " + str(label_softmax[4]) + "분노 " + str(label_softmax[5]) + "상처 ")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [28]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 안녕


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


softmax :  [0.9858453  0.00225786 0.00145339 0.00603477 0.00236074 0.00204795]
>> 입력하신 내용에서 기쁨 느껴집니다.
0.9858453기쁨 0.0022578621불안 0.0014533919당황 0.0060347654슬픔 0.0023607411분노 0.0020479483상처 




KeyboardInterrupt: ignored

# 모델 저장

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/nlp/model_state_dict.pt')

In [ ]:
torch.save(model, '/content/drive/My Drive/nlp/model.pt')

In [ ]:
'''
저 이번주에 결혼해요"

먼저 낚을 생각은 많았지만 그래도 본의에 맞게 살짝쿵 놀라게 해드린 점 사과드린다.

리스본 글쓰기 클럽에서 읽다가 너무 재미져서 허락을 맡고 가져와봤다. 혹시 모르니 이 글은 48시간 뒤에 폭파될 예정.

나는 결혼식이 싫다. 기계적으로 친목회비를 걷어 '00선생님 아들내미 결혼식 축 00만원 지급' 이라는 메신저가 올 때는 몰랐었다. 앞으로 내게 쌓일 (주로) 모바일 청첩장의 축의금은 내가 정해야한다는 사실을. 폐지된 개콘 코너인 애(매한 것) 정(해드리는)남(자) 코너에서 축의금 액수 딱 정해드립니다! 보고 웃는 방청객들이 이해가 가지 않았던 게 엊그제 같은데 벌써 친구 결혼식을 가야 할 나이가 되어버렸다. 이제는 누가 좀 정해줬으면 좋겠다.(개콘은 왜 폐지되서...엉엉)

나는 축의금이 이렇게 사람을 쪼잔하게 만드는 건지 몰랐다. 그간 그 사람과 함께해온 추억과 세월과 정을 숫자로 환산할 수 있다니, 이과들은 물러가라. 문과로서는 도저히 용납할 수 없다. 앞서 말한 것들이 축의금 액수로 환산되어 명부에 적히는 순간, 내 우정도 숫자로 환산될 것만 같은 느낌이다.

만수르라면 고민 없이 신혼부부가 살기좋은 강남 역세권 18평짜리 아파트라도 내놓겠지만 대한민국 20대 사회초년생은 손을 덜덜 떨며 입에 풀칠하기 바쁜 피같은 월급을 쪼개 내야 한다. 그마저도 별 하나에 추억과 별 하나에 이름과 별 하나에 어머니..!!! 하며 너와 나의 우정에 얼만큼의 축의금이 적당한지 밤하늘의 별과 함께 헤아려야한다.

이제 겨우 스타트를 끊은 친구 결혼식이다. 앞으로도 수많은 이들의 결혼식에 가기 위해 축의금 액수를 고민하고, 하객룩을 고민하고 그간의 인간관계에 대해 고민하게 될 생각을 하면 벌써부터 질린다.

물론 이 글을 쓰고도 나는 브라이덜 샤워에 가서 흰 옷을 입고 신부를 빛내주기 위해 이 한 몸 다 바쳐 노력 중일 테다.
